# Web Scraping in Goodreads

This notebook is to get the **Image_url** of all these books that have this feature missing.

### Importing Libraries

In [7]:
import pandas as pd
import numpy as np
import requests # Library used for making HTTP requests
from bs4 import BeautifulSoup # Library for parsing HTML and XML documents
from tqdm import tqdm # To include a progress bar in the loop
import concurrent.futures # To make multiple http requests simultaneously
import time # For time-related functions
import re
import json
from datetime import datetime
from IPython.display import Image # to display images

### Useful Function

Here we define a function that will be used in the loop to make an HTTP request to Goodreads. Then, the year of publication of the book with a given ISBN is searched for.

In [101]:
# This is in order for the request to work.
# It seems that Goodreads blocks the requests made through a script.
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

def get_book_editions(book_workid):
    url = f"https://www.goodreads.com/work/editions/{book_workid}"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        #print(f"Error fetching the page: {response.status_code}")
        return []
    
    soup = BeautifulSoup(response.content, 'html.parser')

    editions = soup.find_all('div', class_='editionData')

    book_editions = []
    for edition in editions:
        edition_info = {}
        title_tag = edition.find('a', class_='bookTitle')
        if title_tag:
            edition_info['title'] = title_tag.text.strip()
            edition_info['link'] = "https://www.goodreads.com" + title_tag['href']
    
        if edition_info:
            book_editions.append(edition_info)

    return book_editions


def is_image_url_accessible(image_url):
    try:
        response = requests.get(image_url)
        if response.status_code == 200:
            return True
        else:
            return False
    except requests.RequestException:
        return False


def get_missing_data(edition_url):
    response = requests.get(edition_url)

    if response.status_code != 200:
        #print(f"Error fetching the page: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    script_tag = soup.find('script', type='application/ld+json') # The image's url is here

    if not script_tag:
        print("No JSON-LD script found")
        return None

    # Convert the JSON fragment into a dictionary
    try:
        book_details = json.loads(script_tag.string)
    except json.JSONDecodeError:
        #print("Error decoding JSON")
        return None

    image_url = book_details.get('image')

    if is_image_url_accessible(image_url) == False:
        return None

    return image_url


def get_data(workid):
    editions = get_book_editions(workid)
    for edition in editions:
        image_url = get_missing_data(edition['link'])
        if image_url:
            return image_url

### Load the dataset

Here we load the books dataset from which we can get the ISBN of the books for which we want to find the genres.

In [102]:
books = pd.read_csv("archive/books.csv")

In [103]:
columns = ['image_url']
for column in columns:
    num_duplicates = books[books[column].duplicated()].shape[0]
    print(f'Number of duplicates in books[{column}]: {num_duplicates}')

Number of duplicates in books[image_url]: 3331


In [104]:
duplicated_images = books[books['image_url'].duplicated()]['image_url'].values
print(set(duplicated_images), '\n')
# The duplicated images correspond all of them to the same picture and it is actually a missing image:
url = duplicated_images[0]
Image(url=url, width=100)

{'https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png'} 



In [105]:
books_missing_data = books[books['image_url'] == url].reset_index(drop=True)

### Web Scraping

What we do is divide the array of work_ids into a number of intervals determined by `divs`. This approach allows us to iterate over these intervals, and further within the work_ids inside each interval, specifying how many intervals we compile each time. This method gives us control over the web scraping, as simply iterating over the entire array of work_ids, i.e., `books['work_id']`, could potentially lead to various issues. Additionally, every time an interval is completed, we store the results in a CSV file to prevent any potential loss of information. 

In [122]:
total = len(books_missing_data['work_id']) # total number of ISBNs
divs = 28 # number of intervals in which we divide the isbns
step = int(total / divs) # length of each interval
ranges = [range(step*i - step, step*i) for i in range(1,divs+1)] # an array with the intervals

key_i = 0 # number of the interval at which we start the for loop
key_f = 28 # number of the interval at which we stop the for loop

#progress_bar = tqdm(total=(key_f-key_i), bar_format='{percentage:.2f}%|{bar}| {n_fmt}/{total_fmt} [{remaining}<{elapsed}, {rate_fmt}]')

for i in range(key_i, key_f): 
    
    workids = books_missing_data['work_id'][ranges[i]] # WorkIDs of the interval i
    
    progress_bar = tqdm(total=step, bar_format='{percentage:.2f}%|{bar}| {n_fmt}/{total_fmt} [{remaining}<{elapsed}, {rate_fmt}]')
    
    data_dict = {
        'WorkID':[],
        'Image_url':[]
    }

    with concurrent.futures.ThreadPoolExecutor() as executor: 
        future_to_workid = {executor.submit(get_data, workid): workid for workid in workids}
        for future in concurrent.futures.as_completed(future_to_workid):
            workid = future_to_workid[future]
            try:
                image_workid = future.result()
                data_dict['WorkID'].append(workid)
                data_dict['Image_url'].append(image_workid)
            except Exception as e:
                data_dict['WorkID'].append(workid)
                data_dict['Image_url'].append(np.nan)
            progress_bar.update(1)
           
    progress_bar.close()

    new_df = pd.DataFrame(data_dict)

    try:
        existing_df = pd.read_csv("books_image_missing.txt", sep="\t")
    except FileNotFoundError:
        existing_df = pd.DataFrame()

    combined_df = pd.concat([existing_df, new_df], ignore_index=True)

    combined_df.to_csv("books_image_missing.txt", sep="\t", index=False)

    time.sleep(2)

    #progress_bar.update(1) 
    
#progress_bar.close()

100.00%|██████████████████████████████████████| 119/119 [00:00<01:39,  1.19it/s]


In [131]:
combined_df[combined_df.isnull().any(axis=1)]

,WorkID,Image_url
117,1180927,NaN
118,1766737,NaN
142,4551869,NaN
170,2247074,NaN
237,3125926,NaN
356,1244564,NaN
427,25704,NaN
713,968512,NaN
754,3898716,NaN
832,1075398,NaN


1. ThreadPoolExecutor: concurrent.futures.ThreadPoolExecutor() creates a pool of threads that can be used to execute multiple calls to the process_isbn function simultaneously. Each thread in the pool will execute a specific task.

2. executor.submit(): executor.submit(process_isbn, isbn) submits a task to the thread pool for execution. The submit function takes two arguments: the function to be executed (process_isbn) and the arguments to be passed to that function (isbn). It returns a Future object that represents the future result of the function call.

3. {executor.submit(process_isbn, isbn): isbn for isbn in isbns}: This is a dictionary comprehension that creates a dictionary where the keys are Future objects returned by executor.submit() and the values are the corresponding ISBNs. This is used to keep track of which ISBN corresponds to which Future.

4. concurrent.futures.as_completed(): This function takes an iterable of Future objects and returns an iterator that yields Future as they are completed. It waits until each Future is completed and then returns the completed Future. In this case, we are passing the future_to_isbn dictionary that contains all the Future objects we have created earlier.

5. for future in concurrent.futures.as_completed(future_to_isbn):: We iterate over the iterator returned by as_completed(). As the Future objects are completed, the loop iterates over them in the order they are completed.

6. isbn = future_to_isbn[future]: Since we are keeping track of which ISBN corresponds to which Future in our future_to_isbn dictionary, we can use the current Future to find its corresponding ISBN.

7. future.result(): future.result() returns the result of the function call associated with the Future. If the function call has not yet finished, result() will block until it is completed. In this case, we are getting the result (i.e., the genre of the book) and storing it in the genres_isbn variable.